### **IMPORTING RELEVANT MODULES**

In [2]:
import numpy as np
import pandas as pd
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### **TOKENIZING AND PREPROCESSING TWEETS**

In [3]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if type(text) != float:
        tokens = word_tokenize(text.lower())
        filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
        return ' '.join(filtered_tokens)
    else:
        return str(text)

df = pd.read_csv('Data/extern.csv')

df['preprocessed_tweet'] = df['tweet'].apply(preprocess_text)
X = df['preprocessed_tweet'].values
y = df['value'].values

### **DIVIDING INTO TRAIN AND VALIDATION SET**

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vocab_size = 50000  # Maximum number of words in the vocabulary
embedding_dim = 100
max_length = 50  # Maximum tweet length


tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

### **CREATION AND TESTING OF THE MODEL**

In [34]:
test_model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='tanh')
])

test_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
test_model.fit(X_train_padded, y_train, epochs=100,batch_size=40,validation_data=(X_test_padded, y_test))

# Evaluate test_model
loss, mae = test_model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}, accuracy: {mae}")

Epoch 1/100
200/200 [==============================] - 119s 345ms/step - loss: 0.6223 - accuracy: 0.3503 - val_loss: 0.6239 - val_accuracy: 0.3375
Epoch 2/100
200/200 [==============================] - 72s 361ms/step - loss: 0.6101 - accuracy: 0.3801 - val_loss: 0.5376 - val_accuracy: 0.5260
Epoch 3/100
200/200 [==============================] - 67s 337ms/step - loss: 0.3580 - accuracy: 0.5656 - val_loss: 0.3608 - val_accuracy: 0.5425
Epoch 4/100
200/200 [==============================] - 68s 337ms/step - loss: 0.1542 - accuracy: 0.6805 - val_loss: 0.3611 - val_accuracy: 0.5980
Epoch 5/100
200/200 [==============================] - 71s 356ms/step - loss: 0.0934 - accuracy: 0.7149 - val_loss: 0.3660 - val_accuracy: 0.5885
Epoch 6/100
200/200 [==============================] - 69s 347ms/step - loss: 0.1419 - accuracy: 0.7055 - val_loss: 0.4496 - val_accuracy: 0.5625
Epoch 7/100
200/200 [==============================] - 67s 336ms/step - loss: 0.1027 - accuracy: 0.6992 - val_loss: 0.3947 

### **SAVING THE MODEL**

In [35]:
test_model.save('Model/test.h5')
print("Model saved successfully!")
tokenizer_filename = "Model/tokenizertest.pkl"
with open(tokenizer_filename, 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)
print("tokenizer saved successfully!")

Model1 saved successfully!
tokenizer saved successfully!
